In [4]:
import numpy as np
import cv2 
# # import scipy
# from PIL import Image

In [5]:
def preprocessing(img) :
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return (img, img_gray)

In [6]:
vdo = cv2.VideoCapture('left_output.avi') ### Video
template_img = cv2.imread('Template-1.png')   ### images
template_img, template_gray = preprocessing(template_img)

In [7]:
sift = cv2.SIFT_create()
bf = cv2.BFMatcher()

In [8]:
### หา good_matches
def feature_object_detection(template_img, template_gray, query_img, query_gray, min_match_number) : 
    template_kpts, template_desc = sift.detectAndCompute(template_gray, None)
    query_kpts, query_desc = sift.detectAndCompute(query_gray, None)
    matches = bf.knnMatch(template_desc, query_desc, k=2)
    good_matches = list()
    good_matches_list = list()
    for m, n in matches :
        if m.distance < 0.8*n.distance :
            good_matches.append(m)
            good_matches_list.append([m])
    
    if len(good_matches) > min_match_number :
        src_pts = np.float32([ template_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
        dst_pts = np.float32([ query_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)

        H, inlier_masks = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0) # H RANSAC
        # get the bounding box around template image
        h, w = template_img.shape[:2]
        template_box = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1,1,2)
        transformed_box = cv2.perspectiveTransform(template_box, H)

        detected_img = cv2.polylines(query_img, [np.int32(transformed_box)], True, (100,255,155), 3, cv2.LINE_AA) ### สร้างเส้นที่ใช้ในการตรวจจับ
        drawmatch_img = cv2.drawMatchesKnn(template_img, template_kpts, detected_img, query_kpts, good_matches_list, None, flags=2, matchesMask=inlier_masks)
        cv2.imshow('frame',detected_img)

        return detected_img, drawmatch_img
    else :
        return
        
#

In [9]:
while vdo.isOpened() :
    ret, frame = vdo.read()
    # y = 252
    # width = 455 - x
    # height = 395 - y
    # term_criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)
    if ret :
        img_to_hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV) 
        upper = np.array([30,255,255]) 
        lower = np.array([20,120,100]) 
        frame_yellow = cv2.inRange(img_to_hsv,lower,upper)
        cv2.imshow('frame_yellow ',frame_yellow)
        
        upper = np.array([10,255,100]) 
        lower = np.array([0,100,50]) 
        frame_red = cv2.inRange(img_to_hsv,lower,upper)
        cv2.imshow('frame_red',frame_red)

        white_colour = np.ones_like(frame_red,np.uint8)*255
        sum_colour = cv2.bitwise_or(frame_yellow,frame_red, mask= white_colour)
        cv2.imshow('sum_colour',sum_colour)

        contours,hierarchys = cv2.findContours(sum_colour ,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)                                                                                            
        back_mask = np.zeros(sum_colour.shape[:2],np.uint8) 
        cv2.imshow(' back_mask',back_mask)
        for X in range(len(contours)):     
            if cv2.contourArea(contours[X])>800 :  
                cv2.drawContours(back_mask,contours,X,(255,255,255),5)
                cv2.drawContours(back_mask,contours,X,(255,255,255),-1)

        contours_mask = cv2.bitwise_or(frame,frame, mask = back_mask)
        contours_mask_gray = cv2.cvtColor(contours_mask, cv2.COLOR_BGR2GRAY)
        cv2.imshow('contours_mask_gray',contours_mask_gray)
        detected, drawmatch =  feature_object_detection(template_img, template_gray, frame, contours_mask_gray, 8)

        # ret, track_window = cv2.CamShift(mask_1_gray, (x, y, width, height), term_criteria)
        # pts = cv2.boxPoints(ret)
        # pts = np.int0(pts)
        # cv2.polylines(frame, [pts], True, (0,255,0), 5)
        # cv2.imshow('mask_1',frame)

        if cv2.waitKey(1) & 0xFF == ord('q') : # this line control the period between image frame
            break
    else :
        break
vdo.release()
cv2.destroyAllWindows()